# Condensación de registros

La idea de esta notebook es condensar los registros de manera tal que quede un registro por usuario.

 - [screen_resolution](#screen_resolution)
 - [cantidad de eventos por categoría](#cantidad-de-eventos-por-categoría)

In [2]:
%run limpieza.ipynb
df = get_clean_df()

Creo el nuevo dataframe que va a tener una fila por usuario:

In [3]:
df_features = pd.DataFrame(index=df['person'].unique())
df_features.index.name = 'person'

In [4]:
df_features.head()

""
person
4886f805
ad93850f
0297fc1e
2d681dd8
cccea85e


#### screen_resolution

Para este atributos se utilizó el promedio entre los distintos valores, sin tener en cuenta los nulos.

In [10]:
df_features['screen_resolution_height'] = df.groupby('person')['screen_resolution_height'].mean()
df_features['screen_resolution_width'] = df.groupby('person')['screen_resolution_width'].mean()

#### cantidad de eventos por categoría

La idea de este feature es tener una columna por categoría que indique la cantidad de eventos realizados por usuario en dicha categoría.

In [17]:
events_per_category_count = df.groupby('person')['event'].value_counts()
events_per_category_count.head()

person    event          
0008ed71  checkout             3
          visited site         2
          generic listing      1
00091926  viewed product     372
          visited site        34
Name: event, dtype: int64

In [22]:
events_per_category = events_per_category_count.unstack().fillna(0)
events_per_category.head()

event,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site
person,,,,,,,,,,,
0008ed71,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0
00091926,15.0,25.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,372.0,34.0
00091a7a,1.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0
000ba417,1.0,24.0,6.0,1.0,14.0,0.0,1.0,0.0,0.0,153.0,6.0
000c79fe,1.0,0.0,1.0,0.0,1.0,0.0,1.0,9.0,0.0,3.0,1.0


In [25]:
df_features = df_features.merge(events_per_category, how='inner', on='person')

Chequeo que no haya habido claves que se hayan quedado fuera del merge:

In [30]:
df_features.shape[0] == df['person'].unique().shape[0]

True

#### cantidad de eventos

In [11]:
df_features['event_count'] = df.groupby('person')['event'].count()

In [26]:
df_features.head()

,screen_resolution_height,screen_resolution_width,event_count,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site
person,,,,,,,,,,,,,,
4886f805,640.0,360.0,9,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,4.0,1.0
ad93850f,640.0,360.0,65,10.0,15.0,1.0,0.0,7.0,0.0,7.0,0.0,0.0,20.0,5.0
0297fc1e,640.0,360.0,567,29.0,4.0,7.0,0.0,21.0,1.0,0.0,6.0,0.0,404.0,95.0
2d681dd8,768.0,1360.0,26,1.0,5.0,1.0,0.0,1.0,0.0,2.0,1.0,0.0,13.0,2.0
cccea85e,864.0,1536.0,836,15.0,7.0,1.0,0.0,20.0,0.0,26.0,1.0,5.0,739.0,22.0
